In [ ]:
import os
from ultralytics import YOLO, trackers
import cv2
import os
import supervision as sv
import cv2
import numpy as np
from ultralytics.data.annotator import auto_annotate
import torch

In [ ]:
import time
import torch
import cv2
import os
from ultralytics import YOLO
import easyocr

import matplotlib.pyplot as plt
reader = easyocr.Reader(['en'])

device = "cuda" if torch.cuda.is_available() else "cpu"

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

model_path = os.path.join('runs/detect/train12/weights', 'best.pt')
model = YOLO(model_path, verbose=False)

input_video_path = './test/small_input.mp4'
output_video_path = './test/output.mp4'

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter(
    output_video_path,
    cv2.VideoWriter_fourcc(*'MP4V'),
    fps,
    (frame_width, frame_height)
)

frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    original_frame = frame.copy()
    
    results = model.predict(frame, device=device, verbose=False)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())
            
            label = f"{model.names[cls]} {conf:.2f}"
            
            croped = original_frame[y1:y2, x1:x2]
            results = reader.readtext(croped, detail=0)
            # plot croped frame with identified text as label
            plt.figure()
            plt.title(f'Text: {''.join(results)}')
            plt.imshow(croped)
            plt.show()
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    frames.append(original_frame)

 
cap.release()
out.release()

In [ ]:
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import requests
import torch
from PIL import Image
from io import BytesIO
import easyocr
import supervision as sv

import cv2
import pytesseract
from matplotlib import pyplot as plt

# Carregar a imagem
image = cv2.imread('./image.png')

from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
)

result = CLIENT.infer(image, model_id="chequemodel/1")

detections = sv.Detections.from_inference(result)

annotated_image = sv.BoundingBoxAnnotator().annotate(image.copy(),detections)
annotated_image = sv.LabelAnnotator(text_scale=0.3).annotate(annotated_image,detections)

sv.plot_image(annotated_image, (6,6))
class_list = detections.data["class_name"]
name_detection = detections[class_list == "Payee_Name"]
name_image = sv.crop_image(image,name_detection.xyxy[0].tolist())

amount_detection = detections[class_list == "Amount_In_Numbers"]
amount_image = sv.crop_image(image,amount_detection.xyxy[0].tolist())

sv.plot_image(name_image,(4,4))
sv.plot_image(amount_image,(2,2))

In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"

psm = 6
config = '--oem 3 --psm %d' % psm


image = frames[0]
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

data = pytesseract.image_to_string(thresh, lang='eng', config=config)
print(data)
# Loop through detected text and draw bounding boxes
for i in range(len(data)):
    x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
    text = data['text'][i]
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Display the result
cv2.imshow('Detected Text', image)
cv2.waitKey(0)
cv2.destroyAllWindows()